In [ ]:
import os
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
#import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dropout

# location of the data
base_dir = '~/'
train_dir = os.path.join(base_dir,'train')
validation_dir = os.path.join(base_dir,'validation')

# Define the subdirectories to include for training and validation
include_subfolders_train = ['aster', 'noaster']
include_subfolders_validation = ['aster', 'noaster']

#for class_subfolders in [include_subfolders_train, include_subfolders_validation]:
#    class_subfolders = [subfolder for subfolder in class_subfolders if exclude_subfolder not in subfolder]


#construct the neural convolution network
model = tf.keras.models.Sequential([

tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(64,64,1)),
tf.keras.layers.MaxPooling2D(2,2),

tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),

tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),

#prepare for the full connection layer
tf.keras.layers.Flatten(),

tf.keras.layers.Dense(512,activation='relu'),
tf.keras.layers.Dropout(0.2),
# sigmoid for 2-body classification
tf.keras.layers.Dense(1,activation='sigmoid')
])

model.summary()

#setting the training 

model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=1e-4),
              metrics=['acc'])

#pre-analyze the data
#train_datagen = ImageDataGenerator(rescale=1./255)
# Add rotation to ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(64,64),
        batch_size=256,
        class_mode='binary',
        color_mode='grayscale',
        classes=include_subfolders_train)

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(64,64),
        batch_size=256,
        class_mode='binary',
        color_mode='grayscale',
        classes=include_subfolders_validation)

#training the model
# Set batch size
#batch_size = 128

history = model.fit_generator(
      train_generator,
      steps_per_epoch=430,
      epochs=100,
      validation_data=validation_generator,
      validation_steps=140,
      verbose=2)

#save the model
model.save("pattern_example_100epoch.h5")